In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplstereonet
import sphstat.singlesample
from sklearn_extra.cluster import KMedoids
import sphstat.singlesample as ss_singlesample
from sphstat.utils import prettyprintdict as ss_prettyprintdict

# initialize
rad = np.pi/180
deg = 180/np.pi

# import data and preview
data_lower = pd.read_csv('frac_attitude_data.csv')
data_lower

In [ ]:
# rename columns
data_lower.rename(columns={'DipDir_': 'dipdir', 'Dip': 'dip'}, inplace=True)

# create new polar coords columns
data_lower['plunge'] = 90 - data_lower['dip']
data_lower['trend'] = data_lower['dipdir'] + 180
data_lower['strike'] = (data_lower['dipdir'] - 90)
# cannot use modulo "%" directly due to precedence in arithmetic operations
data_lower['trend'] = data_lower['trend'] % 360
data_lower['strike'] = data_lower['strike'] % 360

# create new cartesian coords columns for pole
data_lower['l'] = np.cos(data_lower['plunge']*rad) * np.cos(data_lower['trend']*rad)
data_lower['m'] = np.cos(data_lower['plunge']*rad) * np.sin(data_lower['trend']*rad)
data_lower['n'] = -np.sin(data_lower['plunge']*rad)

data_lower

In [ ]:
# duplicate data mirroring l,m,n cartesian poles in the upper hemisphere
data_upper = data_lower.copy()
data_upper['l'] = -data_upper['l']
data_upper['m'] = -data_upper['m']
data_upper['n'] = -data_upper['n']
data_upper

In [ ]:
# concatenate data and data_upper
data_both = pd.concat([data_lower, data_upper])
data_both

In [ ]:
# plot stereonets
fig = plt.figure()
# great circles
ax_1 = fig.add_subplot(131, projection='stereonet')
ax_1.grid()
ax_1.plane(data_lower['strike'] , data_lower['dip'], 'g-', linewidth=.5)
# poles
ax_2 = fig.add_subplot(132, projection='stereonet')
ax_2.grid()
ax_2.line(data_lower['plunge'], data_lower['trend'] , 'g^', markersize=2)
# contours
ax_3 = fig.add_subplot(133, projection='stereonet')
cax = ax_3.density_contourf(data_lower['plunge'], data_lower['trend'] , measurement='lines', cmap='Blues')
ax_3.grid(True)

plt.show()


In [ ]:
# K-medoids clustering
n_clusters = 5
# Specify medoid initialization method with optional init
# array-like of shape(n_clusters, n_features)
# ‘random’ selects n_clusters elements from the dataset
# ‘heuristic’ (default) picks the n_clusters points with the smallest sum distance to every other point
# ‘k-medoids++’ follows an approach based on k-means++_, and in general, gives initial medoids which are more separated than those generated by the other methods
# ‘build’ is a greedy initialization of the medoids used in the original PAM algorithm. Often ‘build’ is more efficient but slower than other initializations on big datasets and it is also very non-robust, if there are outliers in the dataset, use another initialization

# Specify medoid method with method
# ‘alternate’ (default) is faster
# ‘pam’ is more accurate

kmedoids = KMedoids(n_clusters=n_clusters*2, random_state=0, method='pam', init='k-medoids++').fit(data_both[['l', 'm', 'n']])
data_both['cluster'] = kmedoids.labels_
print("medoids:\n", kmedoids.cluster_centers_)
print("data_both['cluster'].unique(): ", data_both['cluster'].unique())
data_both

In [ ]:
# keep clusters with medoid pointing downwards only
cluster_keep = (kmedoids.cluster_centers_[:,2] <= 0).nonzero()[0]
print("cluster_keep: ", cluster_keep)
cluster_keep_centers = kmedoids.cluster_centers_[cluster_keep]
print("cluster_keep_centers:\n", cluster_keep_centers)
data_keep = data_both.loc[data_both['cluster'].isin(cluster_keep)]
print("data_keep['cluster'].unique(): ", data_keep['cluster'].unique())
data_keep

In [ ]:
# plot clusters
fig = plt.figure()
n_subplots = 100 + n_clusters*10
for cluster in data_keep['cluster'].unique():
    n_subplots += 1
    ax = fig.add_subplot(n_subplots, projection='stereonet')
    ax.grid()
    plunge = data_keep[data_keep['cluster'] == cluster]['plunge']
    trend = data_keep[data_keep['cluster'] == cluster]['trend']
    ax.line(plunge, trend, 'g^', markersize=2)

In [ ]:
# spheristat descriptive stats and tests for each cluster
for cluster in data_keep['cluster'].unique():
    print("\n")
    print("cluster: ", cluster)

    # format data for SpheriStat 'cart' (i.e. cartesian)
    sample = dict()
    sample['points'] = data_keep.loc[data_keep["cluster"] == cluster, ["l", "m", "n"]].to_numpy()
    sample['type'] = 'cart'
    sample['n'] = data_keep.loc[data_keep['cluster'] == cluster].shape[1]

    # estimate parametric distributions
    print("-> Fisher distribution parameters:")
    try:
        fisher_dist = ss_singlesample.fisherparams(samplecart=sample, alpha=0.05)
        ss_prettyprintdict(fisher_dist)
    except:
        print("-> Fisher distribution parameters: not available")
    print("-> Kent distribution parameters:")
    try:
        fisher_dist = ss_singlesample.kentparams(samplecart=sample, alpha=0.05)
        ss_prettyprintdict(fisher_dist)
    except:
        print("-> Kent distribution parameters: not available")

    # tests
    print("-> uniform test:")
    try:
        uniform_test = ss_singlesample.isuniform(sample=sample, alpha=0.05)
        ss_prettyprintdict(uniform_test)
    except:
        print("-> uniform test: not available")
    print("-> Is Fisher test:")
    try:
        fisher_test = ss_singlesample.isfisher(samplecart=sample, alpha=0.05)
        ss_prettyprintdict(fisher_test)
    except:
        print("-> Is Fisher test: not available")
    print("-> Is Fisher vs. Kent test:")
    try:
        fisher_kent_test = ss_singlesample.isfishervskent(samplecart=sample, alpha=0.05)
        ss_prettyprintdict(fisher_kent_test)
    except:
        print("-> Is Fisher vs. Kent test: not available")